# Debug Programmatically Built Graphs

This notebook shows how to use the recorder to debug graphs you build from Python code.

In [1]:
from lfx import components as cp
from lfx.graph import Graph
from langflow.debug.recorder import record_graph

/Users/ogabrielluiz/Projects/langflow/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Build a Simple Graph

Let's create a simple flow: ChatInput → Prompt → ChatOutput

In [2]:
# Build graph from components
chat_input = cp.ChatInput()
prompt = cp.PromptComponent()
prompt.set(
    template="You are a helpful assistant. User says: {input}",
    input_text=chat_input.message_response
)
chat_output = cp.ChatOutput().set(input_value=prompt.build_prompt)

graph = Graph(chat_input, chat_output)

print(f"✅ Built graph with {len(graph.vertices)} components")

✅ Built graph with 3 components


## Record the Graph Execution

In [3]:
# Record the graph directly!
recording = await record_graph(graph, flow_name="My Simple Flow")

print(f"Recorded {len(recording.snapshots)} steps")

✅ Recorded 3 component executions
Recorded 3 steps


## Explore the Timeline

In [4]:
recording.show_timeline()


Execution Timeline: My Simple Flow
Method: async_start

Total components: 3
Current position: 0

Timeline:
👉 [  0] ✅ ChatInput
   [  1] ✅ PromptComponent
   [  2] ✅ ChatOutput



## Navigate and Inspect State

In [5]:
# Jump to a component
recording.jump_to("Prompt")
recording.show_state()

✅ Jumped to step 1: PromptComponent

State at Step 1: PromptComponent

Component: PromptComponent-PH6ej
Step: 1

Inputs:
  code: from lfx.base.prompts.api_utils import process_prompt_templa...
  input: ...
  input_text: {
    "text": "",
    "sender": "User",
    "sender_name": "...
  template: You are a helpful assistant. User says: {input}...
  tool_placeholder: ...

Queue at this point:
  - ChatOutput



## View State Deltas

See what changed at each step

In [6]:
# See deltas for each component
for i in range(len(recording.snapshots)):
    recording.jump_to(i)
    snap = recording.snapshots[i]
    delta = snap.get_delta()
    
    comp_name = snap.vertex_id.split("-")[0]
    print(f"\n{i}. {comp_name}")
    
    if "queue" in delta:
        q = delta["queue"]
        if q["added"]:
            print(f"   + Queued: {[v.split('-')[0] for v in q['added']]}")
        if q["removed"]:
            print(f"   - Dequeued: {[v.split('-')[0] for v in q['removed']]}")

✅ Jumped to step 0

0. ChatInput
   + Queued: ['PromptComponent']
✅ Jumped to step 1

1. PromptComponent
   + Queued: ['ChatOutput']
   - Dequeued: ['PromptComponent']
✅ Jumped to step 2

2. ChatOutput
   - Dequeued: ['ChatOutput']


## Advanced Example: Loop Flow

Build a flow with a loop component to see more complex state changes

In [7]:
from lfx.custom import Component
from lfx.io import Output
from lfx.schema.data import Data
from lfx.schema.dataframe import DataFrame

# Create a custom component that generates data
class DataGenerator(Component):
    outputs = [Output(name="output", method="generate")]
    
    def generate(self) -> DataFrame:
        return DataFrame([Data(text=f"Item {i}") for i in range(5)])

# Build loop flow with feedback (like LoopTest.json)
generator = DataGenerator()
loop = cp.LoopComponent()
loop.set(data=generator.generate)

# Downstream processing (item output → parse → back to loop)
parse = cp.ParseDataComponent()
parse.set(data=loop.item_output, template="{text}")

# Feedback: parse result goes back to loop.item input
loop.set(item=parse.parse_data_as_list)

# Final output when loop is done
chat_output = cp.ChatOutput().set(input_value=loop.done_output)

loop_graph = Graph(generator, chat_output)

# Record it
loop_recording = await record_graph(loop_graph, flow_name="Loop Example")

print(f"\nRecorded {len(loop_recording.snapshots)} steps")
loop_recording.show_timeline()

✅ Recorded 11 component executions

Recorded 11 steps

Execution Timeline: Loop Example
Method: async_start

Total components: 11
Current position: 0

Timeline:
👉 [  0] ✅ DataGenerator
   [  1] ✅ LoopComponent
   [  2] ✅ ParseDataComponent
   [  3] ✅ LoopComponent
   [  4] ✅ ParseDataComponent
   [  5] ✅ LoopComponent
   [  6] ✅ ParseDataComponent
   [  7] ✅ LoopComponent
   [  8] ✅ ParseDataComponent
   [  9] ✅ LoopComponent
   [ 10] ❌ ParseDataComponent



## Inspect Loop Iterations

In [8]:
# Find all loop executions
loop_steps = [
    i for i, snap in enumerate(loop_recording.snapshots)
    if "Loop" in snap.vertex_id
]

print(f"Loop executed at steps: {loop_steps}\n")

# Inspect each loop iteration
for i, step in enumerate(loop_steps[:3]):
    loop_recording.jump_to(step)
    snap = loop_recording.snapshots[step]
    delta = snap.get_delta()
    
    print(f"Iteration {i} (step {step}):")
    
    # Show what loop changed
    if "run_predecessors" in delta:
        print(f"   Dependencies modified: {delta['run_predecessors'].keys()}")
    if "run_map" in delta:
        print(f"   run_map modified: {delta['run_map'].keys()}")
    if "queue" in delta:
        print(f"   Queue changes: +{len(delta['queue']['added'])} -{len(delta['queue']['removed'])}")
    
    print()

Loop executed at steps: [1, 3, 5, 7, 9]

✅ Jumped to step 1
Iteration 0 (step 1):
   run_map modified: dict_keys(['LoopComponent-CRpU6', 'DataGenerator-FdMLX'])
   Queue changes: +1 -1

✅ Jumped to step 3
Iteration 1 (step 3):
   run_map modified: dict_keys(['LoopComponent-CRpU6'])
   Queue changes: +1 -1

✅ Jumped to step 5
Iteration 2 (step 5):
   Queue changes: +1 -1



## Save Recording for Later

In [9]:
# Save the recording
loop_recording.save("my_debug_session.pkl")

# Load it later
from langflow.debug.recorder import GraphRecording
loaded = GraphRecording.load("my_debug_session.pkl")

# Continue debugging
loaded.jump_to("Loop")
loaded.show_state()

✅ Saved recording to my_debug_session.pkl


TypeError: ComponentBuildError.__init__() missing 1 required positional argument: 'formatted_traceback'

## Summary

**You can now:**
- ✅ Build graphs programmatically from components
- ✅ Record execution without saving to file
- ✅ Jump to any component and inspect state
- ✅ See state deltas (queue, dependencies) at each step
- ✅ Track loop iterations and progression
- ✅ Save/load debug sessions

**Key API:**
```python
# Record
recording = await record_graph(graph, flow_name="My Flow")

# Navigate
recording.jump_to("ComponentName")  # or step number
recording.next_step()
recording.previous_step()

# Inspect
recording.show_timeline()
recording.show_state()
delta = recording.snapshots[i].get_delta()

# Save
recording.save("debug_session.pkl")
```